In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline 

# 第一次:0.73684

In [2]:
Data_train=pd.read_csv('./titanic/train.csv')

In [3]:
Data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
Data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
null_columns=Data_train.columns[Data_train.isnull().any()]
Data_train[null_columns].isnull().sum()

Age         177
Cabin       687
Embarked      2
dtype: int64

## 缺失值處理:刪掉整個'Cabin' column, 刪掉缺'Age'的row

In [6]:
Data_train=Data_train.drop('Cabin',axis=1)

In [7]:
Data_train=Data_train.drop('Name',axis=1)

In [8]:
Data_train=Data_train.drop('Ticket',axis=1)

In [9]:
Data_train=Data_train.dropna()

In [10]:
Data_train_target=Data_train['Survived']
Data_train_target

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

In [11]:
Data_train=Data_train.drop('Survived',axis=1)
Data_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S


# 將Sex和Embarked欄位元素改成數字

In [12]:
SexClass=Data_train['Sex'].unique()
SexClass

array(['male', 'female'], dtype=object)

In [13]:
Data_train.Sex = pd.Categorical(Data_train.Sex)

In [14]:
Data_train.Sex.cat.codes

0      1
1      0
2      0
3      0
4      1
      ..
885    0
886    1
887    0
889    1
890    1
Length: 712, dtype: int8

In [15]:
Data_train['Sex']=Data_train.Sex.cat.codes
Data_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,S
1,2,1,0,38.0,1,0,71.2833,C
2,3,3,0,26.0,0,0,7.9250,S
3,4,1,0,35.0,1,0,53.1000,S
4,5,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,886,3,0,39.0,0,5,29.1250,Q
886,887,2,1,27.0,0,0,13.0000,S
887,888,1,0,19.0,0,0,30.0000,S
889,890,1,1,26.0,0,0,30.0000,C


In [16]:
Data_train.Embarked = pd.Categorical(Data_train.Embarked)
Data_train['Embarked']=Data_train.Embarked.cat.codes
Data_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,2
1,2,1,0,38.0,1,0,71.2833,0
2,3,3,0,26.0,0,0,7.9250,2
3,4,1,0,35.0,1,0,53.1000,2
4,5,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,886,3,0,39.0,0,5,29.1250,1
886,887,2,1,27.0,0,0,13.0000,2
887,888,1,0,19.0,0,0,30.0000,2
889,890,1,1,26.0,0,0,30.0000,0


## 使用linear regression

In [17]:
from sklearn.model_selection import train_test_split
x=Data_train
y=Data_train_target

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [18]:
lr=LogisticRegression(solver='lbfgs',max_iter=1000)
reg=lr.fit(x_train,y_train)
reg.coef_

array([[ 3.35651852e-04, -1.17616471e+00, -2.54233385e+00,
        -4.70760666e-02, -4.96646124e-01, -7.68111617e-03,
         3.45312140e-03, -1.68387228e-01]])

In [19]:
print("in data accuracy:",reg.score(x_train, y_train))

in data accuracy: 0.8172690763052208


In [20]:
print("validate data accuracy:",reg.score(x_test, y_test))

validate data accuracy: 0.7757009345794392


### 整個Data在train一次

In [21]:
reg=lr.fit(Data_train,Data_train_target)

In [22]:
print("validate data accuracy:",reg.score(x_test, y_test))

validate data accuracy: 0.7850467289719626


## 輸出test data result

In [23]:
Data_test=pd.read_csv('./titanic/test.csv')
Data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [24]:
null_columns=Data_test.columns[Data_test.isnull().any()]
Data_test[null_columns].isnull().sum()

Age       86
Fare       1
Cabin    327
dtype: int64

In [25]:
Data_test=Data_test.drop('Cabin',axis=1)

In [26]:
Data_test=Data_test.drop('Name',axis=1)

In [27]:
Data_test=Data_test.drop('Ticket',axis=1)

In [28]:
Data_test.isnull().any()

PassengerId    False
Pclass         False
Sex            False
Age             True
SibSp          False
Parch          False
Fare            True
Embarked       False
dtype: bool

In [29]:
#Age和Fare包含NaN
Data_test['Age'].fillna(Data_test['Age'].mean(),inplace=True)
Data_test['Fare'].fillna(Data_test['Fare'].mean(),inplace=True)

In [30]:
Data_test.Sex = pd.Categorical(Data_test.Sex)
Data_test['Sex']=Data_test.Sex.cat.codes
Data_test.Embarked = pd.Categorical(Data_test.Embarked)
Data_test['Embarked']=Data_test.Embarked.cat.codes
Data_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.50000,0,0,7.8292,1
1,893,3,0,47.00000,1,0,7.0000,2
2,894,2,1,62.00000,0,0,9.6875,1
3,895,3,1,27.00000,0,0,8.6625,2
4,896,3,0,22.00000,1,1,12.2875,2
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,2
414,1306,1,0,39.00000,0,0,108.9000,0
415,1307,3,1,38.50000,0,0,7.2500,2
416,1308,3,1,30.27259,0,0,8.0500,2


In [31]:
reg.predict(Data_test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [32]:
df1=Data_test['PassengerId'].copy()
df1.reset_index(drop=True, inplace=True)
df2=pd.DataFrame(reg.predict(Data_test))
df2.reset_index(drop=True, inplace=True)
df = pd.concat( [df1, df2], axis=1) 
df.columns=['PassengerId','Survived']
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [33]:
df.to_csv('Result.csv',index=False)

In [34]:
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# 第二次 0.75119

In [35]:
Data_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,2
1,2,1,0,38.0,1,0,71.2833,0
2,3,3,0,26.0,0,0,7.9250,2
3,4,1,0,35.0,1,0,53.1000,2
4,5,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,886,3,0,39.0,0,5,29.1250,1
886,887,2,1,27.0,0,0,13.0000,2
887,888,1,0,19.0,0,0,30.0000,2
889,890,1,1,26.0,0,0,30.0000,0


In [36]:
Data_train_target

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

In [37]:
x=Data_train
y=Data_train_target

x_train,x_valid,y_train,y_valid=train_test_split(x,y,test_size=0.3,random_state=0)

## 標準化

In [38]:
x_train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
202,203,3,1,34.0,0,0,6.4958,2
439,440,2,1,31.0,0,0,10.5000,2
102,103,1,1,21.0,0,1,77.2875,2
118,119,1,1,24.0,0,1,247.5208,0
625,626,1,1,61.0,0,0,32.3208,2
...,...,...,...,...,...,...,...,...
885,886,3,0,39.0,0,5,29.1250,1
239,240,2,1,33.0,0,0,12.2750,2
791,792,2,1,16.0,0,0,26.0000,2
705,706,2,1,39.0,0,0,26.0000,2


### standardization

In [39]:
#standardization
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
x_train_norm = mms.fit_transform(x_train)
x_valid_norm = mms.transform(x_valid)

In [40]:
from sklearn.model_selection import train_test_split
lr=LogisticRegression(solver='lbfgs',max_iter=1000)
reg=lr.fit(x_train_norm,y_train)
reg.coef_

array([[ 0.09087913, -0.993982  , -1.28207385, -0.66929603, -0.48832863,
        -0.01932063,  0.18180579, -0.13739744]])

In [41]:
print("in data accuracy:",reg.score(x_train_norm, y_train))

in data accuracy: 0.8152610441767069


In [42]:
print("validate data accuracy:",reg.score(x_valid_norm, y_valid))

validate data accuracy: 0.7710280373831776


In [43]:
Data_test_norm = mms.fit_transform(Data_test)

In [44]:
df1=Data_test['PassengerId'].copy()
df1.reset_index(drop=True, inplace=True)
df2=pd.DataFrame(reg.predict(Data_test_norm))
df2.reset_index(drop=True, inplace=True)
df = pd.concat( [df1, df2], axis=1) 
df.columns=['PassengerId','Survived']
df.to_csv('Result2.csv',index=False)
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
